In [2]:
import pandas as pd

train = pd.read_csv("labeledTrainData.tsv", header = 0, delimiter = "\t", quoting = 3)
test = pd.read_csv("testData.tsv", header = 0, delimiter = "\t", quoting = 3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header = 0, delimiter = "\t", quoting =3)

# Verify the number of reviews that were read(100,000 in total)
print(train["review"].size, test["review"].size, unlabeled_train["review"].size)

25000 25000 50000


In [3]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False) :
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    
    # 4. Optionally remove stop words(false by default)
    if remove_stopwords :
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    # 5. Retun a list of words
    return(words)

In [4]:
import nltk.data

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False) :
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences= []
    for raw_sentence in raw_sentences :
        if len(raw_sentence) > 0 :
               sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    
    return sentences

In [5]:
sentences = []

for review in train["review"] :
    sentences += review_to_sentences(review, tokenizer)

for review in unlabeled_train["review"] :
    sentences += review_to_sentences(review, tokenizer)

c:\users\na2na\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
c:\users\na2na\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:294: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
c:\users\na2na\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
c:\users\na2na\anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:357: UserWarning: "http://www.archive.org/details/LovefromaStran

In [6]:
print(len(sentences))
print(sentences[1])

795538
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [7]:
import logging

# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

# Set values for various parameters
num_features = 300 # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4 # Number of threads to run in parallel
context = 10 # Context window size
downsampling = 1e-3 # Downsample setting for frequent words

# Initialize and train the model(this will take some time)
from gensim.models import word2vec
model = word2vec.Word2Vec(sentences, workers = num_workers, \
                         size = num_features, min_count = min_word_count, \
                         window = context, sample = downsampling)

model.init_sims(replace=True)

model_name = "300features_40minwords_10context"
model.save(model_name)

2019-10-07 18:38:44,421 : INFO : collecting all words and their counts
2019-10-07 18:38:44,422 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-10-07 18:38:44,461 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-10-07 18:38:44,504 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-10-07 18:38:44,544 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2019-10-07 18:38:44,585 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2019-10-07 18:38:44,627 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2019-10-07 18:38:44,671 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2019-10-07 18:38:44,714 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2019-10-07 18:38:44,759 : INFO : PROGRESS: 

2019-10-07 18:38:47,570 : INFO : PROGRESS: at sentence #710000, processed 15880377 words, keeping 117597 word types
2019-10-07 18:38:47,617 : INFO : PROGRESS: at sentence #720000, processed 16105664 words, keeping 118222 word types
2019-10-07 18:38:47,661 : INFO : PROGRESS: at sentence #730000, processed 16332045 words, keeping 118955 word types
2019-10-07 18:38:47,705 : INFO : PROGRESS: at sentence #740000, processed 16553078 words, keeping 119669 word types
2019-10-07 18:38:47,749 : INFO : PROGRESS: at sentence #750000, processed 16771405 words, keeping 120296 word types
2019-10-07 18:38:47,792 : INFO : PROGRESS: at sentence #760000, processed 16990809 words, keeping 120931 word types
2019-10-07 18:38:47,836 : INFO : PROGRESS: at sentence #770000, processed 17217946 words, keeping 121704 word types
2019-10-07 18:38:47,883 : INFO : PROGRESS: at sentence #780000, processed 17448092 words, keeping 122403 word types
2019-10-07 18:38:47,926 : INFO : PROGRESS: at sentence #790000, processe

2019-10-07 18:39:32,520 : INFO : EPOCH 4 - PROGRESS: at 12.80% examples, 809220 words/s, in_qsize 7, out_qsize 0
2019-10-07 18:39:33,523 : INFO : EPOCH 4 - PROGRESS: at 19.55% examples, 821763 words/s, in_qsize 7, out_qsize 0
2019-10-07 18:39:34,526 : INFO : EPOCH 4 - PROGRESS: at 26.69% examples, 842597 words/s, in_qsize 7, out_qsize 0
2019-10-07 18:39:35,526 : INFO : EPOCH 4 - PROGRESS: at 34.05% examples, 859732 words/s, in_qsize 7, out_qsize 0
2019-10-07 18:39:36,529 : INFO : EPOCH 4 - PROGRESS: at 41.09% examples, 866316 words/s, in_qsize 7, out_qsize 0
2019-10-07 18:39:37,533 : INFO : EPOCH 4 - PROGRESS: at 47.79% examples, 864573 words/s, in_qsize 8, out_qsize 0
2019-10-07 18:39:38,535 : INFO : EPOCH 4 - PROGRESS: at 54.81% examples, 868945 words/s, in_qsize 7, out_qsize 0
2019-10-07 18:39:39,545 : INFO : EPOCH 4 - PROGRESS: at 61.64% examples, 869139 words/s, in_qsize 6, out_qsize 1
2019-10-07 18:39:40,557 : INFO : EPOCH 4 - PROGRESS: at 68.50% examples, 868468 words/s, in_qsiz

In [8]:
def makeFeatureVec(words, model, num_features) :
    # Pre-initialize an empty numpy array(for speed)
    featureVec = np.zeros((num_features,), dtype="float32")
    
    nwords = 0
    
    # Index2word is a list that contains the names of the words in the models's vocabulary
    # Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is inthe model's vocabulary, add its
    # feature vector to the total
    for word in words :
        if word in index2word_set :
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
            
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec
    
def getAvgFeatureVecs(reviews, model, num_features) :
    # Given a set of reviews (each one a list of words), calculate the average feature
    # vector for each one and return a 2D numpy array
    
    # Initialize a counter
    counter = 0
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    
    # Loop through the reviews
    for review in reviews :
        if counter % 1000. == 0. :
            print("Review %d of %d" % (counter , len(reviews)))
        
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        
        # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [9]:
import numpy as np

clean_train_reviews = []
for review in train["review"] :
    clean_train_reviews.append( review_to_wordlist(review, remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

clean_test_reviews = []
for review in test["review"] :
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000


c:\users\na2na\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 220

In [10]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators= 100)

forest = forest.fit(trainDataVecs, train["sentiment"])

result = forest.predict(testDataVecs)

output = pd.DataFrame(data = {"id" : test["id"], "sentiment" : result})
output.to_csv("Word2Vec_AverageVectors.csv", index = False, quoting = 3)

In [16]:
# Clustering
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
print(word_vectors)
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start


[[-0.01829277 -0.09694358 -0.0616742  ... -0.03497111  0.05639735
   0.03080823]
 [ 0.03616401  0.05709035 -0.02230634 ...  0.05877104  0.02723756
  -0.01072849]
 [-0.016651    0.05665086  0.00735852 ... -0.02350433 -0.01187439
  -0.1044334 ]
 ...
 [-0.05627724  0.09490713  0.05373332 ...  0.08109144  0.00635513
  -0.03727254]
 [ 0.0360207  -0.00842665 -0.02338804 ...  0.12595339 -0.01606896
   0.0267111 ]
 [-0.02158932  0.00666285  0.03328843 ... -0.06737489 -0.0070851
  -0.04615021]]


c:\users\na2na\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':


AttributeError: 'KMeans' object has no attribute 'wv'